# KGTK Browser Cache Setup

This note book will create the SQLite DB Cache and the required indices for KGTK Browser.

The required input parameters are:
- input_path: Path where the following files should be present
    1. labels.en.tsv.gz
    2. aliases.en.tsv.gz
    3. descriptions.en.tsv.gz
    4. claims.tsv.gz
    5. metadata.property.datatypes.tsv.gz
    6. qualifiers.tsv.gz
    7. metadata.pagerank.undirected.tsv.gz
    8. class-visualization.edge.tsv.gz **# optional, required for class visualization graph**
    9. class-visualization.node.tsv.gz **# optional, required for class visualization graph**
    10. derived.isastar.tsv.gz **# This file is required for creating the ES index only.**

The files `metadata.pagerank.undirected.tsv.gz` and `derived.isastar.tsv.gz` are created by running [this](https://github.com/usc-isi-i2/kgtk/blob/dev/use-cases/Wikidata%20Useful%20Files.ipynb) notebook
- output_path: Output path 
- project_name: folder inside the `output_path` where the required files and cache will be created
- es_host: ES host, default `localhost`
- es_port: ES port, default `9200`
- es_index: name of the desired elasticsearch index, default `wikidata-dwd-kgtk-search-01`
- create_db: variable to control creation of sqlite database cache. `yes|no`, default `yes`
- create_es: variable to control creation of the ES index. `yes|no`, defaut `no`
- create_class_viz: variable to control creation of class visualization tables in the sqlite database cache. Most probably, if you are not working with Wikidata, you will not have files # 8 and 9 listed above. Set this parameter to 'no' in that case. `yes|no`, default `yes`

**Cache file location:** `<output_path>/<project_name>/temp.<project_name>/wikidata.sqlite3.db`

In [1]:
import os
import pandas as pd
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
import kgtk.kypher.api as kapi
from kgtk.functions import kgtk, kypher

In [2]:
input_path = "/data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file"
output_path = "/data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file"

project_name = "kgtk-browser-files"

files = 'label,pagerank_undirected,alias,description,claims,datatypes,qualifiers'

es_host = "http://localhost"
es_port = 9200
es_index = "wikidata-dwd-kgtk-search-01"

create_db = 'yes'
create_es = 'no'
create_class_viz = 'yes'

In [3]:
files = files.split(',')

create_sqlite_cache = create_db.lower() == 'yes'
create_es_index = create_es.lower() == 'yes'
create_class_viz_tables = create_class_viz.lower() == 'yes'

additional_files = {
    'classvizedge': 'class-visualization.edge.tsv.gz',
    'classviznode': 'class-visualization.node.tsv.gz'
}

In [4]:
ck = ConfigureKGTK(files)
if create_class_viz_tables:
    ck.configure_kgtk(input_graph_path=input_path,
                      output_path=output_path,
                      project_name=project_name,
                     additional_files=additional_files)
else:
    ck.configure_kgtk(input_graph_path=input_path,
                      output_path=output_path,
                      project_name=project_name)

User home: /nas/home/amandeep
Current dir: /data/amandeep/github/kgtk-browser
KGTK dir: /data/amandeep/github
Use-cases dir: /data/amandeep/github/use-cases


In [5]:
ck.print_env_variables()

kgtk: kgtk
STORE: /data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/kgtk-browser-files/temp.kgtk-browser-files/wikidata.sqlite3.db
OUT: /data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/kgtk-browser-files
GRAPH: /data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file
EXAMPLES_DIR: /data/amandeep/github/examples
USE_CASES_DIR: /data/amandeep/github/use-cases
TEMP: /data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/kgtk-browser-files/temp.kgtk-browser-files
kypher: kgtk query --graph-cache /data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/kgtk-browser-files/temp.kgtk-browser-files/wikidata.sqlite3.db
KGTK_GRAPH_CACHE: /data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/kgtk-browser-files/temp.kgtk-browser-files/wikidata.sqlite3.db
KGTK_LABEL_FILE: /data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/labels.en.tsv.gz
KGTK_OPTION_DEBUG: false
label: /data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/labels.en.tsv.gz
pageran

## Load the files into cache

In [6]:
if create_sqlite_cache:
    ck.load_files_into_cache()

kgtk query --graph-cache /data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/kgtk-browser-files/temp.kgtk-browser-files/wikidata.sqlite3.db -i "/data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/labels.en.tsv.gz" --as label  -i "/data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/metadata.pagerank.undirected.tsv.gz" --as pagerank_undirected  -i "/data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/aliases.en.tsv.gz" --as alias  -i "/data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/descriptions.en.tsv.gz" --as description  -i "/data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/claims.tsv.gz" --as claims  -i "/data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/metadata.property.datatypes.tsv.gz" --as datatypes  -i "/data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/qualifiers.tsv.gz" --as qualifiers  -i "/data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/class-visualization.edge.tsv.gz" --as classvizedge  -i "/d

## Define the Kypher API

In [7]:
_kapi2 = kapi.KypherApi(graphcache=os.environ['STORE'], loglevel=1, index='auto',
                      maxresults=100, maxcache=0)

## Create a file with `label`, `undirected_pagerank` and `description`

In [8]:
if create_sqlite_cache:
    !kgtk query --gc $STORE \
        -i label pagerank_undirected description\
        --match 'label: (qnode)-[l]->(y), pagerank: (qnode)-[:Pundirected_pagerank]->(pr)' \
        --opt 'description: (qnode)-[:description]->(d)' \
        --return 'qnode as node1, l.label as label, y as node2, upper(y) as `node2;upper`, pr as `node1;pagerank`, ifnull(d, "") as `node1;description`' \
        --order-by 'qnode' \
        -o $OUT/label_pagerank_undirected_description.tsv.gz

### Load this file into cache as well

In [9]:
if create_sqlite_cache:
    !kgtk query --gc $STORE -i $OUT/label_pagerank_undirected_description.tsv.gz --as l_d_pgr_ud --limit 10

node1	label	node2	node2;upper	node1;pagerank	node1;description
P10	label	'video'@en	'VIDEO'@EN	1.9177407615866818e-08	'relevant video. For images, use the property P18. For film trailers, qualify with \"object has role\" (P3831)=\"trailer\" (Q622550)'@en
P1000	label	'record held'@en	'RECORD HELD'@EN	1.4103331270065626e-08	'notable record achieved by a person or entity, include qualifiers for dates held'@en
P10000	label	'Research Vocabularies Australia ID'@en	'RESEARCH VOCABULARIES AUSTRALIA ID'@EN	2.586054821150431e-08	'Identifier of a vocabulary in Research Vocabularies Australia'@en
P10001	label	'Austrian Football Association player ID'@en	'AUSTRIAN FOOTBALL ASSOCIATION PLAYER ID'@EN	3.152198892346769e-08	'identifier for an association football player on the Austrian Football Association website'@en
P10002	label	'Dewan Negara ID'@en	'DEWAN NEGARA ID'@EN	2.3820406824266033e-08	'numerical identifier for members or former members of the Dewan Negara of Malaysia on the official website'@

## Create the required indices

In [10]:
%%time 
if create_sqlite_cache:
    !kgtk --debug query -i l_d_pgr_ud --idx node1 "node2;upper" label text:node2//name=ldpgridx --gc $STORE --limit 5

[2022-05-16 20:23:59 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_10 AS graph_10_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 20:23:59 sqlstore]: CREATE INDEX "graph_10_node1_idx" ON "graph_10" ("node1")
[2022-05-16 20:24:27 sqlstore]: ANALYZE "graph_10_node1_idx"
[2022-05-16 20:24:30 sqlstore]: CREATE INDEX "graph_10_node2;upper_idx" ON "graph_10" ("node2;upper")
[2022-05-16 20:25:59 sqlstore]: ANALYZE "graph_10_node2;upper_idx"
[2022-05-16 20:26:04 sqlstore]: CREATE INDEX "graph_10_label_idx" ON "graph_10" ("label")
[2022-05-16 20:26:27 sqlstore]: ANALYZE "graph_10_label_idx"
[2022-05-16 20:26:30 sqlstore]: CREATE VIRTUAL TABLE "graph_10_txtidx_ldpgridx" USING FTS5 ("node2", tokenize="trigram", content="graph_10")
[2022-05-16 20:26:30 sqlstore]: INSERT INTO "graph_10_txtidx_ldpgridx" ("node2") SELECT "node2" FROM graph_10
node1	label	node2	node2;upper	node1;pagerank	node1;description
P10	la

In [11]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i label --idx label --gc $STORE --limit 5

[2022-05-16 20:36:08 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 20:36:08 sqlstore]: CREATE INDEX "graph_1_label_idx" ON "graph_1" ("label")
[2022-05-16 20:36:31 sqlstore]: ANALYZE "graph_1_label_idx"
id	node1	label	node2	lang	rank	node2;wikidatatype
P10-label-en	P10	label	'video'@en	en		
P1000-label-en	P1000	label	'record held'@en	en		
P10000-label-en	P10000	label	'Research Vocabularies Australia ID'@en	en		
P10001-label-en	P10001	label	'Austrian Football Association player ID'@en	en		
P10002-label-en	P10002	label	'Dewan Negara ID'@en	en		
CPU times: user 673 ms, sys: 158 ms, total: 831 ms
Wall time: 38.2 s


In [12]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i alias --idx label --gc $STORE --limit 5

[2022-05-16 20:36:41 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_3 AS graph_3_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 20:36:41 sqlstore]: CREATE INDEX "graph_3_label_idx" ON "graph_3" ("label")
[2022-05-16 20:36:50 sqlstore]: ANALYZE "graph_3_label_idx"
id	node1	label	node2	lang	rank	node2;wikidatatype
P10-alias-en-282226-0	P10	alias	'gif'@en	en		
P10-alias-en-2f86d8-0	P10	alias	'animation'@en	en		
P10-alias-en-c1427e-0	P10	alias	'media'@en	en		
P10-alias-en-c61ab1-0	P10	alias	'trailer (Commons)'@en	en		
P10000-alias-en-0df7f5-0	P10000	alias	'Australian Research Vocabularies ID'@en	en		
CPU times: user 282 ms, sys: 62 ms, total: 344 ms
Wall time: 15 s


In [13]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i description --idx id --gc $STORE --limit 5

[2022-05-16 20:36:55 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_4 AS graph_4_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 20:36:55 sqlstore]: CREATE INDEX "graph_4_id_idx" ON "graph_4" ("id")
[2022-05-16 20:37:27 sqlstore]: ANALYZE "graph_4_id_idx"
id	node1	label	node2	lang	rank	node2;wikidatatype
P10-description-en	P10	description	'relevant video. For images, use the property P18. For film trailers, qualify with \"object has role\" (P3831)=\"trailer\" (Q622550)'@en	en		
P1000-description-en	P1000	description	'notable record achieved by a person or entity, include qualifiers for dates held'@en	en		
P10000-description-en	P10000	description	'Identifier of a vocabulary in Research Vocabularies Australia'@en	en		
P10001-description-en	P10001	description	'identifier for an association football player on the Austrian Football Association website'@en	en		
P10002-description-en	P10002	description	'

In [14]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i claims --idx label node1 node2 id --gc $STORE --limit 5

[2022-05-16 20:37:41 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_5 AS graph_5_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 20:37:41 sqlstore]: CREATE INDEX "graph_5_label_idx" ON "graph_5" ("label")
[2022-05-16 20:53:10 sqlstore]: ANALYZE "graph_5_label_idx"
[2022-05-16 20:53:50 sqlstore]: CREATE INDEX "graph_5_node1_idx" ON "graph_5" ("node1")
[2022-05-16 21:04:13 sqlstore]: ANALYZE "graph_5_node1_idx"
[2022-05-16 21:04:57 sqlstore]: CREATE INDEX "graph_5_node2_idx" ON "graph_5" ("node2")
[2022-05-16 21:19:44 sqlstore]: ANALYZE "graph_5_node2_idx"
[2022-05-16 21:20:35 sqlstore]: CREATE INDEX "graph_5_id_idx" ON "graph_5" ("id")
[2022-05-16 21:30:20 sqlstore]: ANALYZE "graph_5_id_idx"
node1	label	node2	id	lang	rank	node2;wikidatatype
Q1	Pshort_abstract	"The universe (Latin: universus) is all of space and time and their contents, including planets, stars, galaxies, and all other forms of matt

In [15]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i datatypes --idx label node1 --gc $STORE --limit 5

[2022-05-16 21:31:58 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_6 AS graph_6_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 21:31:58 sqlstore]: CREATE INDEX "graph_6_label_idx" ON "graph_6" ("label")
[2022-05-16 21:31:58 sqlstore]: ANALYZE "graph_6_label_idx"
[2022-05-16 21:31:58 sqlstore]: CREATE INDEX "graph_6_node1_idx" ON "graph_6" ("node1")
[2022-05-16 21:31:58 sqlstore]: ANALYZE "graph_6_node1_idx"
id	node1	label	node2	lang	rank	node2;wikidatatype
P10-datatype	P10	datatype	commonsMedia			
P1000-datatype	P1000	datatype	wikibase-item			
P10000-datatype	P10000	datatype	external-id			
P10001-datatype	P10001	datatype	external-id			
P10002-datatype	P10002	datatype	external-id			
CPU times: user 466 ms, sys: 112 ms, total: 577 ms
Wall time: 26.1 s


In [16]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i qualifiers --idx node2 node1 label --gc $STORE --limit 5

[2022-05-16 21:32:03 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_7 AS graph_7_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 21:32:03 sqlstore]: CREATE INDEX "graph_7_node2_idx" ON "graph_7" ("node2")
[2022-05-16 21:38:16 sqlstore]: ANALYZE "graph_7_node2_idx"
[2022-05-16 21:38:31 sqlstore]: CREATE INDEX "graph_7_node1_idx" ON "graph_7" ("node1")
[2022-05-16 21:41:45 sqlstore]: ANALYZE "graph_7_node1_idx"
[2022-05-16 21:42:08 sqlstore]: CREATE INDEX "graph_7_label_idx" ON "graph_7" ("label")
[2022-05-16 21:44:41 sqlstore]: ANALYZE "graph_7_label_idx"
node1	label	node2	id	lang	rank	node2;wikidatatype
P10-P1855-Q15075950-7eff6d65-0	P10	"Smoorverliefd 12 september.webm"	P10-P1855-Q15075950-7eff6d65-0-P10-54b214-0			commonsMedia
P10-P1855-Q15075950-7eff6d65-0	P3831	Q622550	P10-P1855-Q15075950-7eff6d65-0-P3831-Q622550-0			wikibase-item
P10-P1855-Q4504-a69d2c73-0	P10	"Komodo dragons video.ogv"	P10-P

In [17]:
%%time
if create_sqlite_cache and create_class_viz_tables:
    !kgtk --debug query -i classvizedge --idx label --gc $STORE --limit 5

[2022-05-16 21:45:07 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_8 AS graph_8_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 21:45:07 sqlstore]: CREATE INDEX "graph_8_label_idx" ON "graph_8" ("label")
[2022-05-16 21:47:32 sqlstore]: ANALYZE "graph_8_label_idx"
node1	label	node2	graph	edge_type
Q10267817	P279	Q18553442	Q1225194	subclass
Q107715	P279	Q309314	Q246672	subclass
Q107715	P279	Q309314	Q937228	subclass
Q107715	P279	Q309314	Q7184903	subclass
Q107715	P279	Q309314	Q35120	subclass
CPU times: user 3.03 s, sys: 678 ms, total: 3.71 s
Wall time: 2min 50s


In [18]:
%%time
if create_sqlite_cache and create_class_viz_tables:
    !kgtk --debug query -i classvizedge --idx index:graph --gc $STORE --limit 5

[2022-05-16 21:47:54 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_8 AS graph_8_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 21:47:54 sqlstore]: CREATE INDEX "graph_8_graph_idx" ON "graph_8" ("graph")
[2022-05-16 21:50:04 sqlstore]: ANALYZE "graph_8_graph_idx"
node1	label	node2	graph	edge_type
Q10267817	P279	Q18553442	Q1225194	subclass
Q107715	P279	Q309314	Q246672	subclass
Q107715	P279	Q309314	Q937228	subclass
Q107715	P279	Q309314	Q7184903	subclass
Q107715	P279	Q309314	Q35120	subclass
CPU times: user 2.7 s, sys: 504 ms, total: 3.21 s
Wall time: 2min 33s


In [19]:
%%time
if create_sqlite_cache and create_class_viz_tables:
    !kgtk --debug query -i classviznode --idx label --gc $STORE --limit 5

[2022-05-16 21:50:27 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_9 AS graph_9_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 21:50:27 sqlstore]: CREATE INDEX "graph_9_label_idx" ON "graph_9" ("label")
[2022-05-16 21:54:10 sqlstore]: ANALYZE "graph_9_label_idx"
node1	graph	instance_count	node_type	label	tooltip
Q10267817	Q1225194	1	many_subclasses	'autosomal recessive disease'@en	autosomal recessive disease (Q10267817)<BR/>instance count: 1<BR/>node type: many_subclasses
Q10267817	Q18553442	1	many_subclasses	'autosomal recessive disease'@en	autosomal recessive disease (Q10267817)<BR/>instance count: 1<BR/>node type: many_subclasses
Q107715	Q246672	95	many_subclasses	'physical quantity'@en	physical quantity (Q107715)<BR/>instance count: 95<BR/>node type: many_subclasses
Q107715	Q937228	95	many_subclasses	'physical quantity'@en	physical quantity (Q107715)<BR/>instance count: 95<BR/>node type: man

In [20]:
%%time
if create_sqlite_cache and create_class_viz_tables:
    !kgtk --debug query -i classviznode --idx index:graph --gc $STORE --limit 5

[2022-05-16 21:54:29 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_9 AS graph_9_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-05-16 21:54:29 sqlstore]: CREATE INDEX "graph_9_graph_idx" ON "graph_9" ("graph")
[2022-05-16 21:56:13 sqlstore]: ANALYZE "graph_9_graph_idx"
node1	graph	instance_count	node_type	label	tooltip
Q10267817	Q1225194	1	many_subclasses	'autosomal recessive disease'@en	autosomal recessive disease (Q10267817)<BR/>instance count: 1<BR/>node type: many_subclasses
Q10267817	Q18553442	1	many_subclasses	'autosomal recessive disease'@en	autosomal recessive disease (Q10267817)<BR/>instance count: 1<BR/>node type: many_subclasses
Q107715	Q246672	95	many_subclasses	'physical quantity'@en	physical quantity (Q107715)<BR/>instance count: 95<BR/>node type: many_subclasses
Q107715	Q937228	95	many_subclasses	'physical quantity'@en	physical quantity (Q107715)<BR/>instance count: 95<BR/>node type: man

## Take a look at cache file content

In [21]:
!kgtk query --gc $STORE --show-cache 

Graph Cache:
DB file: /data/amandeep/wikidata-20211027-dwd-v3/browser-claims-file/kgtk-browser-files/temp.kgtk-browser-files/wikidata.sqlite3.db
  size:  215.73 GB   	free:  0 Bytes   	modified:  2022-05-16 21:56:22

KGTK File Information:
alias:
  size:  161.03 MB   	modified:  2022-05-16 16:54:40   	graph:  graph_3
claims:
  size:  11.58 GB   	modified:  2022-05-16 18:43:32   	graph:  graph_5
classvizedge:
  size:  586.90 MB   	modified:  2022-04-20 22:12:31   	graph:  graph_8
classviznode:
  size:  489.17 MB   	modified:  2022-04-20 22:12:48   	graph:  graph_9
datatypes:
  size:  50.75 KB   	modified:  2022-05-16 16:56:23   	graph:  graph_6
description:
  size:  393.31 MB   	modified:  2022-05-16 16:56:18   	graph:  graph_4
l_d_pgr_ud:
  size:  1.27 GB   	modified:  2022-05-16 20:22:05   	graph:  graph_10
label:
  size:  637.08 MB   	modified:  2022-05-16 16:53:57   	graph:  graph_1
pagerank_undirected:
  size:  740.37 MB   	modified:  2022-04-20 22:16:46   	graph:  graph_2
qualifie

## Define a function to do a `textmatch` search

In [22]:
def text_search_labels(search_text, limit=20):
    text_search_labels_query = _kapi2.get_query(
        doc="Doc string here",
        name=f"text_search_labels_{search_text}",
        inputs='l_d_pgr_ud',
        match='l_d_pgr_ud: (qnode)-[l:label]->(y)',
        where=f'textmatch(y, "{search_text}")',
       ret='distinct qnode as node1, y as label, 10*matchscore(y) as score, cast(l.`node1;pagerank`, float) as prank, l.`node1;description` as description',
       order='score*prank',
       limit=limit
    )
    results =  list([list(x) for x in text_search_labels_query.execute()])
    df = pd.DataFrame(results, columns=['node1', 'label', 'score', 'pagerank', 'description'])
    print(len(df))
    return df
    

In [23]:
text_search_labels('arn sch')

[2022-04-21 08:19:06 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_10_c1."node1" "_aLias.node1", graph_10_c1."node2" "_aLias.label", (? * BM25(txtidx_1.graph_10_txtidx_ldpgridx)) "_aLias.score", CAST(graph_10_c1."node1;pagerank" AS float) "_aLias.prank", graph_10_c1."node1;description" "_aLias.description"
     FROM graph_10 AS graph_10_c1, graph_10_txtidx_ldpgridx AS txtidx_1
     WHERE graph_10_c1."label" = ?
        AND txtidx_1."node2" MATCH ? and txtidx_1.rowid = graph_10_c1.rowid
     ORDER BY ("_aLias.score" * "_aLias.prank") ASC
     LIMIT ?
  PARAS: [10, 'label', 'arn sch', 20]
---------------------------------------------


20


,node1,label,score,pagerank,description
0,Q154770,'Arnold Schoenberg'@en,-107.405619,2.983258e-07,'Austrian-American composer (1874-1951)'@en
1,Q2685,'Arnold Schwarzenegger'@en,-100.426118,2.794892e-07,"'Austrian-American actor, businessman, bodybui..."
2,Q668095,'Scharnstein'@en,-119.905570,1.907455e-07,'municipality in Austria'@en
3,Q506494,'Schäftlarn'@en,-122.277359,1.862084e-07,'municipality of Germany'@en
4,Q723261,'Scharn'@en,-132.783438,1.588464e-07,"'neighborhood in Maastricht, Netherlands'@en"
5,Q664826,'Scharnitz'@en,-124.744872,1.239395e-07,'municipality in Austria'@en
6,Q522320,'Arnschwang'@en,-122.277359,1.133833e-07,'municipality of Germany'@en
7,Q5659790,'Harnischia'@en,-122.277359,1.014908e-07,'genus of insects'@en
8,Q2667816,'Scharnegoutum'@en,-115.427713,1.051748e-07,'human settlement in the Netherlands'@en
9,Q77325,'Arno Schmidt'@en,-117.624040,9.288887e-08,'German poet'@en


## Define a function to search for Qnodes Exactly

In [24]:
def exact_search_items(search_text, limit=20):
    search_text = search_text.upper()
    text_search_labels_query =  _kapi2.get_query(
    doc="""
    Create the Kypher query used by 'BrowserBackend.get_node_labels()'
    for case_independent searches.
    Given parameters 'NODE' and 'LANG' retrieve labels for 'NODE' in
    the specified language (using 'any' for 'LANG' retrieves all labels).
    Return distinct 'node1', 'node_label' pairs as the result (we include
    'NODE' as an output to make it easier to union result frames).
    """,
    name=f'exact_search_items{search_text}',
    inputs='l_d_pgr_ud',
    match='l_d_pgr_ud: (n)-[r:label]->(l)',
    where=f'n="{search_text}"',
    ret='distinct n as node1, l as node_label, r.`node1;description` as description',
)
    results =  list([list(x) for x in text_search_labels_query.execute()])
    df = pd.DataFrame(results, columns=['node1', 'label', 'description'])
    print(len(df))
    return df
    

In [25]:
%%time
exact_search_items('q30')

[2022-04-21 08:19:22 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_10_c1."node1" "_aLias.node1", graph_10_c1."node2" "_aLias.node_label", graph_10_c1."node1;description" "_aLias.description"
     FROM graph_10 AS graph_10_c1
     WHERE graph_10_c1."label" = ?
        AND (graph_10_c1."node1" = ?)
     LIMIT ?
  PARAS: ['label', 'Q30', 100]
---------------------------------------------


1
CPU times: user 182 ms, sys: 44.9 ms, total: 227 ms
Wall time: 935 ms


,node1,label,description
0,Q30,'United States of America'@en,'sovereign state in North America'@en


In [26]:
%%time
exact_search_items('Q140')

[2022-04-21 08:19:22 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_10_c1."node1" "_aLias.node1", graph_10_c1."node2" "_aLias.node_label", graph_10_c1."node1;description" "_aLias.description"
     FROM graph_10 AS graph_10_c1
     WHERE graph_10_c1."label" = ?
        AND (graph_10_c1."node1" = ?)
     LIMIT ?
  PARAS: ['label', 'Q140', 100]
---------------------------------------------


1
CPU times: user 30.8 ms, sys: 6.17 ms, total: 37 ms
Wall time: 54.8 ms


,node1,label,description
0,Q140,'lion'@en,'species of big cat'@en


## Define a function to search labels Exactly

In [27]:
def exact_search_labels(search_text, limit=20):
    search_text = f"'{search_text.upper()}'@EN"
    text_search_labels_query =  _kapi2.get_query(
    doc="""
     Exact Match case insensitive query
    """,
    name=f'exact_search_labels{search_text}',
    inputs='l_d_pgr_ud',
    match=f'l_d_pgr_ud: (n)-[r:label]->(l)',
    where=f'r.`node2;upper`="{search_text}"',
    ret='distinct n as node1, l as node_label, cast("-1.0", float) as score, cast(r.`node1;pagerank`, float) as prank, r.`node1;description` as description',
    order='score*prank',
    limit=limit
)
    results =  list([list(x) for x in text_search_labels_query.execute()])
    df = pd.DataFrame(results, columns=['node1', 'label', 'score', 'prank', 'description'])
    print(len(df))
    return df

In [28]:
exact_search_labels('canada')

[2022-04-21 09:54:48 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_10_c1."node1" "_aLias.node1", graph_10_c1."node2" "_aLias.node_label", CAST(? AS float) "_aLias.score", CAST(graph_10_c1."node1;pagerank" AS float) "_aLias.prank", graph_10_c1."node1;description" "_aLias.description"
     FROM graph_10 AS graph_10_c1
     WHERE graph_10_c1."label" = ?
        AND (graph_10_c1."node2;upper" = ?)
     ORDER BY ("_aLias.score" * "_aLias.prank") ASC
     LIMIT ?
  PARAS: ['-1.0', 'label', "'CANADA'@EN", 20]
---------------------------------------------


20


,node1,label,score,prank,description
0,Q16,'Canada'@en,-1.0,1.052877e-03,'sovereign state in North America'@en
1,Q2569593,'Canada'@en,-1.0,8.219015e-08,'French colony in New France'@en
2,Q13265725,'Canada'@en,-1.0,2.816562e-08,'family name'@en
3,Q106501608,'Canada'@en,-1.0,1.475482e-08,'Vanity Fair caricature of The Rt Hon Sir Wilf...
4,Q13265795,'Canada'@en,-1.0,1.402956e-08,'2012 novel by Richard Ford'@en
5,Q2608363,'Canada'@en,-1.0,1.286213e-08,"'village in Appelscha, in the Netherlands'@en"
6,Q18612153,'Canada'@en,-1.0,1.278788e-08,'180th strip of the webcomic xkcd'@en
7,Q257304,'Canada'@en,-1.0,1.266340e-08,'Wikimedia disambiguation page'@en
8,Q103921530,'Canada'@en,-1.0,1.258331e-08,"'Shipwreck off the Scottish Coast, imported fr..."
9,Q14624136,'Canada'@en,-1.0,1.196157e-08,'moth genus of Pteromalidae'@en


## Define a function to fo a `textlike` search

In [29]:
def text_like_search_labels(search_text, limit=20):
    search_label = f"%{'%'.join(search_text.split(' '))}%"
    print(search_text)
    text_search_labels_query = _kapi2.get_query(
        doc="Doc string here",
        name=f"text_like_search_labels_{search_text}",
        inputs='l_d_pgr_ud',
        match='l_d_pgr_ud: (qnode)-[l:label]->(y)',
        where=f'textlike(y, "{search_label}")',
       ret='distinct qnode as node1, y as label, matchscore(y) as score, cast(l.`node1;pagerank`, float) as prank, l.`node1;description` as description',
       order='score*prank',
       limit=limit
    )
    results =  list([list(x) for x in text_search_labels_query.execute()])
    df = pd.DataFrame(results, columns=['node1', 'label', 'score', 'pagerank', 'description'])
    print(len(df))
    return df
    

In [30]:
text_like_search_labels("fifa group b")

[2022-04-21 09:54:55 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_10_c1."node1" "_aLias.node1", graph_10_c1."node2" "_aLias.label", BM25(txtidx_1.graph_10_txtidx_ldpgridx) "_aLias.score", CAST(graph_10_c1."node1;pagerank" AS float) "_aLias.prank", graph_10_c1."node1;description" "_aLias.description"
     FROM graph_10 AS graph_10_c1, graph_10_txtidx_ldpgridx AS txtidx_1
     WHERE graph_10_c1."label" = ?
        AND txtidx_1."node2" LIKE ? and txtidx_1.rowid = graph_10_c1.rowid
     ORDER BY ("_aLias.score" * "_aLias.prank") ASC
     LIMIT ?
  PARAS: ['label', '%fifa%group%b%', 20]
---------------------------------------------


fifa group b
20


,node1,label,score,pagerank,description
0,Q31189406,'2018 FIFA World Cup Group B'@en,-14.908334,3.970111e-08,
1,Q10260332,'2014 FIFA World Cup Group B'@en,-14.908334,2.652110e-08,
2,Q17115977,'Category:2014 FIFA World Cup group table temp...,-11.121773,2.265241e-08,'Wikimedia category'@en
3,Q18608402,'Category:2015 FIFA Women\'s World Cup group t...,-10.116338,1.991870e-08,'Wikimedia category'@en
4,Q20730977,'2018 FIFA World Cup qualification – UEFA Grou...,-11.372956,1.526389e-08,
5,Q187411,'2010 FIFA World Cup Group B'@en,-14.908334,1.102419e-08,'football tournament'@en
6,Q601980,'2014 FIFA World Cup qualification – UEFA Grou...,-11.372956,1.365065e-08,
7,Q39134591,'Template:2018 FIFA World Cup Group B table'@en,-12.199530,1.267818e-08,'Wikimedia template'@en
8,Q39134633,'Template:2018 FIFA World Cup Group C table'@en,-12.199530,1.266194e-08,'Wikimedia template'@en
9,Q39395683,'Template:2018 FIFA World Cup Group D table'@en,-12.199530,1.264030e-08,'Wikimedia template'@en


# Create the JSON lines file required for KGTK search Elasticsearch Index

## Concatenate all the input files

In [ ]:
if create_es_index:
    kgtk(""" cat 
        -i "$GRAPH/labels.en.tsv.gz"
        -i "$GRAPH/aliases.en.tsv.gz"
        -i "$GRAPH/descriptions.en.tsv.gz"
        -i "$GRAPH/claims.tsv.gz"
        -i "$GRAPH/qualifiers.tsv.gz"
        -i "$GRAPH/metadata.pagerank.undirected.tsv.gz"
        -i "$GRAPH/derived.isastar.tsv.gz"
        -o "$OUT"/kgtk.search.unsorted.tsv.gz""")

## Sort

In [ ]:
if create_es_index:
    kgtk(f"""sort --column node1  
        -X "--parallel 8 --buffer-size 60% --temporary-directory {os.environ['TEMP']}" 
        -o "$OUT/kgtk.search.sorted.tsv.gz" """)

## Run `kgtk build-kgtk-search-input` command to create the json lines file

In [ ]:
if create_es_index:
    kgtk("""build-kgtk-search-input --input-file "$OUT/kgtk.search.sorted.tsv.gz"
            --output-file "$OUT"/kgtk.search.sorted.jl 
            --label-properties label 
            --alias-properties alias 
            --extra-alias-properties P1448,P1705,P1477,P1810,P742,P1449 
            --description-properties description 
            --pagerank-properties Pundirected_pagerank 
            --mapping-file "$OUT"/wikidata_es_kgtk_search_mapping.json 
            --property-datatype-file "$GRAPH"/metadata.property.datatypes.tsv.gz""")

## Create the Elasticsearch Index

In [ ]:
if create_es_index:
    os.environ['ESURL'] = f"{es_host}:{es_port}"
    os.environ['ESINDEX'] = es_index

In [ ]:
if create_es_index:
    !curl -H "Content-Type: application/json" -XPUT $ESURL/$ESINDEX -d @"$OUT/wikidata_es_kgtk_search_mapping.json"

## Load the json lines file to the ES Index

The following command requires [table-linker](https://github.com/usc-isi-i2/table-linker) to be installed in the current virtual environment.

In [ ]:
if create_es_index:
    !tl load-elasticsearch-index --es-url $ESURL --es-index $ESINDEX --es-version 7 --kgtk-jl-path "$OUT"/kgtk.search.sorted.jl